In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Step 2: Upload dataset from Google Drive
from google.colab import files
# You can upload your file manually
uploaded = files.upload()

# Step 3: Load dataset
df = pd.read_csv('CVD_cleaned.csv')  # replace filename if different

# Step 4: Data exploration
print("First few rows:\n", df.head())
print("\nData info:\n", df.info())
print("\nMissing values:\n", df.isnull().sum())

# --- Step 1: Define features and target ---
target_column = 'General_Health'
X = df.drop(target_column, axis=1) # Start with original df for features
Y = df[target_column] # Select target from original df

# --- Step 2: Encode categorical features ---
# Identify categorical columns excluding the target column
categorical_cols_features = X.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical features
X_encoded = pd.get_dummies(X, columns=categorical_cols_features, drop_first=True)

# --- Step 3: Encode target variable if categorical ---
# Check if the target variable needs encoding
if Y.dtype == 'object':
    le = LabelEncoder()
    Y_encoded = le.fit_transform(Y)
else:
    # If the target is already numerical, use it directly
    Y_encoded = Y

# --- Step 4: Split data into train and test sets ---
# Use the encoded features (X_encoded) and encoded target (Y_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_encoded, Y_encoded, test_size=0.2, random_state=42, stratify=Y_encoded # Use Y_encoded for stratify
)

# --- Step 5: Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Model 1: SVM ---
svm_clf = SVC()
svm_clf.fit(X_train_scaled, Y_train)
Y_pred_svm = svm_clf.predict(X_test_scaled)
svm_accuracy = accuracy_score(Y_test, Y_pred_svm)
print("SVM Accuracy:", svm_accuracy)
print("SVM Classification Report:\n", classification_report(Y_test, Y_pred_svm))
# Plot confusion matrix
cm = confusion_matrix(Y_test, Y_pred_svm)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('SVM Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# --- Model 2: Random Forest ---
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train_scaled, Y_train)
Y_pred_rf = rf_clf.predict(X_test_scaled)
rf_accuracy = accuracy_score(Y_test, Y_pred_rf)
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Classification Report:\n", classification_report(Y_test, Y_pred_rf))
# Plot confusion matrix
cm = confusion_matrix(Y_test, Y_pred_rf)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
plt.title('Random Forest Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# --- Model 3: Logistic Regression ---
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(X_train_scaled, Y_train)
Y_pred_lr = lr_clf.predict(X_test_scaled)
lr_accuracy = accuracy_score(Y_test, Y_pred_lr)
print("Logistic Regression Accuracy:", lr_accuracy)
print("Logistic Regression Classification Report:\n", classification_report(Y_test, Y_pred_lr))
# Plot confusion matrix
cm = confusion_matrix(Y_test, Y_pred_lr)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges')
plt.title('Logistic Regression Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# --- Accuracy comparison bar chart ---
models = ['SVM', 'Random Forest', 'Logistic Regression']
accuracies = [svm_accuracy, rf_accuracy, lr_accuracy]

plt.figure(figsize=(8,6))
sns.barplot(x=models, y=accuracies)
for i, v in enumerate(accuracies):
    plt.text(i, v + 0.01, f"{v:.2f}", ha='center', fontsize=12)
plt.ylim(0, 1)
plt.ylabel('Accuracy')
plt.title('Model Accuracy Comparison')
plt.show()

# --- Sample prediction with a random test sample ---
sample_idx = np.random.choice(X_test_scaled.shape[0]) # Use X_test_scaled for index
sample_features_scaled = X_test_scaled[sample_idx].reshape(1, -1) # Get scaled sample and reshape

print("Sample input features (scaled):")
print(sample_features_scaled) # Print scaled features for clarity

# Predictions
svm_pred = svm_clf.predict(sample_features_scaled)[0]
rf_pred = rf_clf.predict(sample_features_scaled)[0]
lr_pred = lr_clf.predict(sample_features_scaled)[0]

# Decode predictions if target was encoded
# Check if le exists before trying to use it
if 'le' in locals() and isinstance(le, LabelEncoder):
    svm_pred_decoded = le.inverse_transform([svm_pred])[0]
    rf_pred_decoded = le.inverse_transform([rf_pred])[0]
    lr_pred_decoded = le.inverse_transform([lr_pred])[0]

    print(f"\nPredicted health status:")
    print(f"SVM Prediction: {svm_pred_decoded}")
    print(f"Random Forest Prediction: {rf_pred_decoded}")
    print(f"Logistic Regression Prediction: {lr_pred_decoded}")
else:
     # If target was not encoded, print the numerical predictions
    print(f"\nPredicted health status (numerical):")
    print(f"SVM Prediction: {svm_pred}")
    print(f"Random Forest Prediction: {rf_pred}")
    print(f"Logistic Regression Prediction: {lr_pred}")

Saving CVD_cleaned.csv to CVD_cleaned.csv
First few rows:
   General_Health                  Checkup Exercise Heart_Disease Skin_Cancer  \
0           Poor  Within the past 2 years       No            No          No   
1      Very Good     Within the past year       No           Yes          No   
2      Very Good     Within the past year      Yes            No          No   
3           Poor     Within the past year      Yes           Yes          No   
4           Good     Within the past year       No            No          No   

  Other_Cancer Depression Diabetes Arthritis     Sex Age_Category  \
0           No         No       No       Yes  Female        70-74   
1           No         No      Yes        No  Female        70-74   
2           No         No      Yes        No  Female        60-64   
3           No         No      Yes        No    Male        75-79   
4           No         No       No        No    Male          80+   

   Height_(cm)  Weight_(kg)    BMI Smoking_Hi